### 2
+pip install missingno

In [1]:
# Importing required libraries
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import missingno as msno

import warnings
warnings.filterwarnings('ignore')


#setting display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# loading data
main_csv = pd.read_csv('loan.csv',encoding = "ISO-8859-1", low_memory=False)

In [ ]:
main_csv.head(5)

In [ ]:
print(main_csv.shape)

In [ ]:
main_csv.describe()

In [ ]:
# checking data types
daypes= main_csv.dtypes
daypes

# DATA Celaning

In [ ]:
msno.matrix(main_csv)

In [ ]:
# Visualize missing values as a matrix Before removing Missing values columns

plt.figure(figsize=(10,8))
sns.heatmap(main_csv.isnull(), cbar=False)
plt.title("Missing Value Heatmap")
plt.show()

In [ ]:
plt.figure(figsize=(26, 10))
plt.bar(main_csv.columns, main_csv.isnull().sum())
plt.xlabel("Columns")
plt.ylabel("Number of Missing Values")
plt.title("Missing Values per Column")
plt.xticks(rotation=90)
plt.show()

In [ ]:
msno.bar(main_csv)

In [ ]:
# Visualize the correlation between the number of
# missing values in different columns as a heatmap
msno.heatmap(main_csv.select_dtypes('number'))

In [ ]:
# Drop columns not useful for analysis (Customer behavior variables)

main_csv = main_csv.drop(['delinq_2yrs', 'inq_last_6mths', 'open_acc','pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp',
                  'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
                  'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d','collections_12_mths_ex_med', 'chargeoff_within_12_mths', 'tax_liens','id', 'member_id',
                                 'url', 'desc', 'zip_code','pub_rec_bankruptcies','earliest_cr_line'
                 ], axis = 1)

In [ ]:
main_csv.shape

In [ ]:
#checking empty columns in percentage
round((main_csv.isnull().sum())*100/len(main_csv.index))

In [ ]:
#drop all the columns which contains more then 55 % missing Data 
missing_values_colms = main_csv.columns[(main_csv.isnull().sum()*100/len(main_csv.index)) > 55]
print(missing_values_colms)
main_csv=main_csv.drop(missing_values_colms,axis=1)

In [ ]:
main_csv.head()

In [ ]:
main_csv.shape

In [ ]:
msno.bar(main_csv)

In [ ]:
msno.matrix(main_csv)

In [ ]:
#Rechecking empty columns in percentage
round((100*main_csv.isnull().sum())/len(main_csv.index)) 

In [ ]:
main_csv.nunique()

In [ ]:
main_csv=main_csv = main_csv.drop(['delinq_amnt','acc_now_delinq','application_type','policy_code','initial_list_status','pymnt_plan'], axis = 1)

In [ ]:
main_csv.shape

In [ ]:
main_csv.head(20)

#### remove all the rows which one contained loan status is current becuase its not make any sens for loan defaulter.It still going on so we not have past data to making decision

In [ ]:
main_csv=main_csv[main_csv['loan_status'] !='Current']

In [ ]:
main_csv.shape

#### Now we able to see that only 71 columns and 1340973 rows available for analysis

In [ ]:
main_csv['loan_status'].value_counts()

In [ ]:
# ploting a count plt for loan status for two varaibles Fully Paid and Charged Off
sns.countplot(data=main_csv,x=main_csv['loan_status'])
plt.title('Loan_status count plot')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Completed loans are those which are either charged off
main_csv=main_csv[(main_csv['loan_status'] =='Fully Paid')|(main_csv['loan_status'] =='Charged Off')]

In [ ]:
# ploting a count plt for loan status for two varaibles Fully Paid and Charged Off
sns.countplot(data=main_csv,x=main_csv['loan_status'])
plt.title('Loan_status count plot')
plt.xticks(rotation=90)
plt.show()

In [ ]:
main_csv.shape

In [ ]:
# matrix plot after removing current values 
msno.matrix(main_csv)

In [ ]:
main_csv.dtypes

In [ ]:
main_csv.head()

In [ ]:
#converting issue date to date time
from datetime import datetime

def convert_to_datetime(date_str):
    """Converts a string in the format 'Month-Year' to a datetime object.

    Args:
        date_str: The input string in the format 'Month-Year'.

    Returns:
        A datetime object representing the first day of the given month and year.
    """

    try:
        # Parse the date string using a specific format
        dt_obj = datetime.strptime(date_str, '%b-%Y')
        # Set the day to the first day of the month
        dt_obj = dt_obj.replace(day=1)
        return dt_obj
    except ValueError:
        print("Invalid date format. Please use 'Month-Year' format.")
        return None

# Example usage:
main_csv['issue_d'] = main_csv['issue_d'].apply(convert_to_datetime)

In [ ]:
main_csv.head()

#### Deriverd variables

In [ ]:
# creating two derived columns for month and year
main_csv['month'] = main_csv['issue_d'].dt.month
main_csv['year'] = main_csv['issue_d'].dt.year

In [ ]:
main_csv.head()

In [ ]:
print(main_csv['year'].value_counts())

In [ ]:
# ploting a count plt for loan status for two varaibles Fully Paid and Charged Off
sns.countplot(data=main_csv,x=main_csv['year'])
plt.title('loan issued year count plot')
plt.xlabel('years')
plt.ylabel('Total numbers of loans')
plt.show()

##### In above counter plot we able to seen that In 2015 they approved highest loans and in 2007 lowest loan the series of year no. of loan issued- 2007 < 2008 < 2009 < 2010 < 2011 < 2018 < 2012 < 2013 < 2017 < 2014 < 2016 < 2018

In [ ]:
# updating data of employ length
main_csv["emp_length"]= main_csv["emp_length"].str.replace("10+ years", "10")
main_csv["emp_length"]= main_csv["emp_length"].str.replace("< 1 year", "0")
main_csv["emp_length"]= main_csv["emp_length"].str.replace("years", "")
main_csv["emp_length"]= main_csv["emp_length"].str.replace("year", "")
main_csv["emp_length"]= main_csv["emp_length"].str.replace("+", "")

In [ ]:
main_csv["title"].fillna("NA", inplace=True)
main_csv["emp_title"].fillna("NA", inplace=True)

In [ ]:
main_csv['home_ownership'].value_counts()

In [ ]:
# dropping the rows of home_ownership which have NONE values-
main_csv = main_csv[main_csv['home_ownership'] != 'NONE']

In [ ]:
main_csv.info()

In [ ]:
# converting object data type into numeric data type
cols = ['loan_amnt','funded_amnt','funded_amnt_inv','installment','annual_inc','dti','emp_length']
main_csv[cols] = main_csv[cols].apply(pd.to_numeric)

In [ ]:
msno.matrix(main_csv)

In [ ]:
print(main_csv['emp_title'].value_counts().head(50))

In [ ]:
value_counts = main_csv['emp_title'].value_counts()
value_counts.to_csv('emp_title_counts.csv')

In [ ]:
emp_title = pd.read_csv('emp_title_counts.csv')
emp_title.shape

In [ ]:
# After analysing Data of title and emp_title able to analyse its not usable so droping these two columns-
main_csv=main_csv = main_csv.drop(['title','emp_title'], axis = 1)

In [ ]:
main_csv.shape

In [ ]:
main_csv.isnull().sum()

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)  # Adjust n_neighbors as needed

imputer.fit(main_csv.select_dtypes('number'))

df_imputed = imputer.transform(main_csv.select_dtypes('number'))

df_imputed = pd.DataFrame(df_imputed, columns=main_csv.select_dtypes('number').columns)

In [ ]:
for col in main_csv.select_dtypes('number'):
    main_csv[col] = main_csv[col].fillna(main_csv[col].median())

In [ ]:
main_csv.isnull().sum()

# Outliers treatmernt

#### for numerical datatypes variable

In [ ]:
main_csv.info()

In [ ]:
main_csv.head()

In [ ]:
main_csv.isnull().sum()

In [ ]:
# Check for empty DataFrame
if len(main_csv) == 0:
    print("main_csv DataFrame is empty!")
    # Handle empty DataFrame (e.g., exit or populate with data)

# Check data type of loan_amnt
if not pd.api.types.is_numeric_dtype(main_csv['loan_amnt']):
    try:
        main_csv['loan_amnt'] = pd.to_numeric(main_csv['loan_amnt'], errors='coerce')
    except:
        print("Error converting loan_amnt to numeric")
        # Handle conversion error (e.g., clean data or skip plotting)

# Reset index (optional, if index mismatch is suspected)
main_csv = main_csv.reset_index(drop=True)

# Try creating the boxplot again
for col in main_csv.select_dtypes('int').columns:
    sns.boxplot(main_csv[col])
    plt.show()  # Display the plot

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'main_csv' is your DataFrame containing numerical columns
for col in main_csv.select_dtypes(include='float64'):  # Specify data type explicitly
    sns.boxplot(x=col, data=main_csv)  # Set 'x' as the variable for boxplot
    plt.title(f"Distribution of {col}")  # Add informative title
    plt.xlabel(col)  # Add label for x-axis
    plt.ylabel("Value")  # Add label for y-axis
    plt.show()

plt.tight_layout()  # Adjust spacing between subplots (after all plots are created)

In [ ]:
main_csv.head()

In [ ]:
main_csv.isnull().sum()

In [ ]:
main_csv.info()

In [ ]:
annual_inc95 = main_csv['annual_inc'].quantile(0.95)
main_csv = main_csv[main_csv['annual_inc'] <= annual_inc95]

In [ ]:

dti95 = main_csv['dti'].quantile(0.95)
main_csv = main_csv[main_csv['dti'] <= dti95]

# Univariate Analysis

Univariate numerical variable analysis
loan_amnt

funded_amnt

funded_amnt_inv

int_rate

installment

annual_inc

emp_length

dti

month

year

In [ ]:
#setting figure size of plot
plt.figure(figsize=(20,4))

#subplot index
plt.subplot(1,3,1)

#Distribution plot
sns.distplot(main_csv['loan_amnt'])

#subplot index
plt.subplot(1,3,2)

#Distribution plot
sns.distplot(main_csv['funded_amnt'])

#subplot index
plt.subplot(1,3,3)

#Distribution plot
sns.distplot(main_csv['funded_amnt_inv'])
plt.show()

In [ ]:
# count plot for emp_length and 

print(main_csv['emp_length'].value_counts())
plt.figure(figsize=(20,4))
plt.subplot(1,2,1)

sns.countplot(data=main_csv,x=main_csv['emp_length'])
plt.ylabel('Total numbers of loans')

plt.subplot(1,2,2)

sns.countplot(data=main_csv,x='emp_length',hue='loan_status')
plt.suptitle('loan issued emp_length count plot')

plt.ylabel('Total numbers of loans')
plt.show()

### Insights-
More then 10 years experience employee have majority borrower

More thne 10 yeras employee charged off in majority

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd  # Assuming 'main_csv' is a pandas DataFrame

# Check DTI range (optional)
print("Minimum DTI:", main_csv['dti'].min())
print("Maximum DTI:", main_csv['dti'].max())

# First plot (distribution of dti)
sns.displot(data=main_csv, x='dti', bins=15, kde=True)

# Adjust x-axis ticks if necessary based on data range
data_min = main_csv['dti'].min()
data_max = main_csv['dti'].max()
adjusted_ticks = range(int(data_min), int(data_max + 1), 5)  # Round to nearest integers
plt.xticks(adjusted_ticks)

plt.title('Loan Issued DTI Distribution')
plt.xlabel('DTI')
plt.ylabel('Total Number of Loans')
plt.grid(True)
plt.show()

# Second plot (distribution of dti by loan status)
sns.displot(data=main_csv, x='dti', hue='loan_status', bins=15, kde=True)

# No need to adjust ticks here (reuse adjusted_ticks if necessary)
# plt.xticks(adjusted_ticks)  # Uncomment if needed

plt.title('Loan Issued DTI Distribution by Loan Status')
plt.ylabel('Total Number of Loans')
plt.show()

### InsightsMajority of borrowers have greter debit respect to monthly earnings for dti ratio in between 9 to 30 and those are also in majority of charged off.k

In [ ]:
# distribution plot for loan_amnt and 
sns.displot(data=main_csv,x=main_csv['loan_amnt'],bins=15,kde=True)
plt.title('loan issued loan_amnt dis plot')
plt.xlabel('loan_amnt')
plt.ylabel('Total numbers of loans')
plt.grid(True);
plt.show()

sns.displot(data=main_csv,x='loan_amnt',hue='loan_status',bins=15,kde=True)
plt.title('loan issued loan_amnt dis plot')
plt.ylabel('Total numbers of loans')
plt.show()

### Insights
Majority of borrowers applied for loan amount between 5k to 18k majority of charged off which applied for loan amount 4k to 12k

In [ ]:
# distribution plot for installment and 
plt.figure(figsize=(15,4))

sns.displot(data=main_csv,x=main_csv['installment'],bins=15,kde=True)
plt.title('loan issued installment dis plot')
plt.xlabel('installment')
plt.ylabel('Total numbers of loans')
plt.grid(True);
plt.show()




sns.displot(data=main_csv,x='installment',hue='loan_status',bins=15,kde=True)
plt.title('loan issued installment dis plot')
plt.ylabel('Total numbers of loans')
plt.grid(True)
plt.show()

### Insights
From Installment data, we can say that most of them have taken loan who paying installment inbetween 200 to 650.

Charged Off loan status is also high for same range(200-650)

In [ ]:
# count plot for year and 

print(main_csv['year'].value_counts())
plt.figure(figsize=(15,4))
plt.subplot(1,2,1)

sns.countplot(data=main_csv,x=main_csv['year'])
plt.ylabel('Total numbers of loans')

plt.subplot(1,2,2)

sns.countplot(data=main_csv,x='year',hue='loan_status')
plt.ylabel('Total numbers of loans')

plt.suptitle('loan issued year count plot')
plt.grid(True)
plt.show()

### Insights
From year wise data, we can say that most of them have taken loan in year 2015 and its increasing from 2007 to 2015 continuously and showing a decline after 2015 till 2018

Charged Off loan status is also high for same year

In [ ]:
# count plot for month and


print(main_csv['month'].value_counts())
plt.figure(figsize=(15,4))
plt.subplot(1,2,1)

sns.countplot(data=main_csv,x=main_csv['month'])
plt.ylabel('Total numbers of loans')

plt.subplot(1,2,2)

sns.countplot(data=main_csv,x='month',hue='loan_status')
plt.ylabel('Total numbers of loans')

plt.suptitle('loan issued month count plot')
plt.grid(True)
plt.show()

### Insights
From monthly wise data, we can say that most of them have taken loan in year's last quarter.

Charged Off loan status is also high for last quarter

In [ ]:
# distribution plot for int_rate and 
sns.displot(data=main_csv,x=main_csv['int_rate'],bins=20,kde=True)
plt.title('loan issued int_rate dis plot')
plt.ylabel('Total numbers of loans')
plt.grid(True);
plt.show()


sns.displot(data=main_csv,x='int_rate',hue='loan_status',bins=15,kde=True)
plt.title('loan issued int_rate dis plot')
plt.ylabel('Total numbers of loans')
plt.grid(True)
plt.show()

### Insights
Borrowers no. increased between 5 to 12 but its going down in between 12 to 30. And same followings for charged off.

In [ ]:
# distribution plot for annual_inc and 
sns.displot(data=main_csv,x=main_csv['annual_inc'],bins=20,kde=True)
plt.title('loan issued annual_inc dis plot')
plt.xlabel('annual_inc')
plt.ylabel('Total numbers of loans')
plt.grid(True);
plt.show()

sns.displot(data=main_csv,x='annual_inc',hue='loan_status',bins=15,kde=True)
plt.title('loan issued annual_inc dis plot')
plt.ylabel('Total numbers of loans')
plt.grid(True)
plt.show()

### Insights
Most of people taking loan which have lower(30k-80k) income and these chances have most probably to charged off repect to higher annual income

# unordered categorical data analysis
loan term

In [ ]:
# count plot for term and
print(main_csv['term'].value_counts())
plt.figure(figsize=(15,4))
plt.subplot(1,2,1)

sns.countplot(data=main_csv,x=main_csv['term'])
plt.ylabel('Total numbers of loans')

plt.subplot(1,2,2)

sns.countplot(data=main_csv,x='term',hue='loan_status')
plt.ylabel('Total numbers of loans')

plt.suptitle('loan issued term count plot')
plt.grid(True)
plt.show()

### Insights
majority of borrowers taken loan for 36 months and most chances of charged off is taken loan for 60 months

# orderd categorical Data analysis
loan_status purpose home ownership state grade subgrade

In [ ]:
# count plot for loan_status and
sns.countplot(data=main_csv,x=main_csv['loan_status'])
plt.title('loan issued loan_status count plot')
plt.xlabel('loan_status')
plt.ylabel('Total numbers of loans')
plt.show()

### Insights
- More then 800000 persond fully paid the loan

- More then 200000 charged off

In [ ]:
# count plot for purpose and

print(main_csv['purpose'].value_counts())
plt.figure(figsize=(20,7))
plt.subplot(1,2,1)

sns.countplot(data=main_csv,y=main_csv['purpose'])
plt.xlabel('Total numbers of loans')

plt.subplot(1,2,2)

sns.countplot(data=main_csv,y='purpose',hue='loan_status')
plt.xlabel('Total numbers of loans')

plt.suptitle('loan issued purpose count plot')
plt.grid(True)
plt.show()

### Insights
- Majority of borrowers loan apply for debt_consolation and paying bils of credi_card

- Majority of charged off have from debt_consolation

In [ ]:
# count plot for addr_state and

#print(main_csv['addr_state'].value_counts())
plt.figure(figsize=(20,15))
plt.subplot(1,2,1)

sns.countplot(data=main_csv,y=main_csv['addr_state'])
plt.xlabel('Total numbers of loans')

plt.subplot(1,2,2)

sns.countplot(data=main_csv,y='addr_state',hue='loan_status')
plt.xlabel('Total numbers of loans')

plt.suptitle('loan issued addr_state count plot')
plt.grid(True)
plt.show()

### Insights
- Majority of borrowers belongs to large urban cities like california, new york, texas, florida and majority of charged off have from california, new york etc.

In [ ]:
# count plot for home_ownership and
print(main_csv['home_ownership'].value_counts())
plt.figure(figsize=(10,7))
plt.subplot(1,2,1)

sns.countplot(data=main_csv,x=main_csv['home_ownership'])
plt.ylabel('Total numbers of loans')
plt.xticks(rotation=90, ha='right')

plt.subplot(1,2,2)

sns.countplot(data=main_csv,x='home_ownership',hue='loan_status')
plt.ylabel('Total numbers of loans')
plt.xticks(rotation=90, ha='right')


plt.suptitle('loan issued home_ownership count plot')
plt.grid(True)
plt.show()

### Insights
- Majority of pepoles taking loan is did't have ownership of house they coming from rent and Mortage category.

- Charged Off loan status is also high for same catrgries (Rent and Mortage)

In [ ]:
# count plot for grade and
print(main_csv['grade'].value_counts())
plt.figure(figsize=(15,7))
plt.subplot(1,2,1)

sns.countplot(data=main_csv,x=main_csv['grade'])
plt.ylabel('Total numbers of loans')

plt.subplot(1,2,2)

sns.countplot(data=main_csv,x='grade',hue='loan_status')
plt.ylabel('Total numbers of loans')

plt.suptitle('loan issued grade count plot')
plt.grid(True)
plt.show()

### Insights
- we can say that most of them have taken loan is coming from grade B C A D E and F.

- But charged Off loan status is high for grade C D B E and A.

In [ ]:
# count plot for sub_grade and
print(main_csv['sub_grade'].value_counts())
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)

sns.countplot(data=main_csv,x=main_csv['sub_grade'])
plt.ylabel('Total numbers of loans')
plt.xticks(rotation=90, ha='right')

plt.subplot(1,2,2)

sns.countplot(data=main_csv,x='sub_grade',hue='loan_status')
plt.ylabel('Total numbers of loans')
plt.xticks(rotation=90, ha='right')

plt.suptitle('loan issued sub_grade count plot')
plt.grid(True)
plt.show()

In [ ]:
# Assuming 'main_csv' is a pandas DataFrame

# Get sub-grade counts in descending order
sub_grade_counts = main_csv['sub_grade'].value_counts().sort_values(ascending=False)

# First plot (total count)
plt.figure(figsize=(20, 5))
plt.subplot(1, 2, 1)

sns.countplot(data=main_csv, x=main_csv['sub_grade'], order=sub_grade_counts.index)
plt.ylabel('Total Numbers of Loans')
plt.xticks(rotation=90, ha='right')

# Second plot (count by loan status)
plt.subplot(1, 2, 2)

sns.countplot(data=main_csv, x='sub_grade', hue='loan_status', order=sub_grade_counts.index)
plt.ylabel('Total Numbers of Loans')
plt.xticks(rotation=90, ha='right')

plt.suptitle('Loan Issued Sub-Grade Count Plot')
plt.grid(True)
plt.show()

### Insights
looking subgrade data, we can say that most of them have taken loan is coming under C1 B4 B3 B5 C2 B3 C3 etc. Charged Off loan status is also high for same sub-grades.

In [ ]:
# count plot for verification_status
print(main_csv['verification_status'].value_counts())
plt.figure(figsize=(15,7))
plt.subplot(1,2,1)

sns.countplot(data=main_csv,x=main_csv['verification_status'])
plt.ylabel('Total numbers of loans')

plt.subplot(1,2,2)

sns.countplot(data=main_csv,x='verification_status',hue='loan_status')
plt.ylabel('Total numbers of loans')

plt.suptitle('loan issued verification_status count plot')
plt.grid(True)
plt.show()

### Insights
mostly 50% borrowers verified or source verified and majority of chargedoff happen also from the same category

# Creating new dataframe for charged_off

In [ ]:
# Derived columns
main_csv['loan_amnt_cats'] = pd.cut(main_csv['loan_amnt'], [0, 7000, 14000, 21000, 28000, 35000], labels=['0-7000', '7000-14000', '14000-21000', '21000-28000', '28000 +'])

# categorise annual incomes into buckets which will help in analysis further in bivariate analysis.
main_csv['annual_inc_cats'] = pd.cut(main_csv['annual_inc'], [0, 20000, 40000, 60000, 80000,1000000], labels=['0-20000', '20000-40000', '40000-60000', '60000-80000', '80000 +'])

# categorise intrest rates into buckets which will help in analysis further in bivariate analysis.
main_csv['int_rate_cats'] = pd.cut(main_csv['int_rate'], [0, 10, 12.5, 16, 20], labels=['0-10', '10-13', '12.5-16', '16 +'])

# categorise dti into buckets for bivariate analysis.
main_csv['dti_cats'] = pd.cut(main_csv['dti'], [0, 5, 10, 15, 20, 25], labels=['0-5', '05-10', '10-15', '15-20', '25+'])

In [ ]:
charged_off_main_csv=main_csv.loc[main_csv['loan_status']=='Charged Off']

In [ ]:
charged_off_main_csv.head()

In [ ]:
charged_off_main_csv.shape

In [ ]:
charged_off_main_csv.info()

# Univariate Segmented categorial analysis
doing Analysis with new data frame charged_off data taken

In [ ]:
charged_off_main_csv.select_dtypes('int').head()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=charged_off_main_csv,x='home_ownership',hue='purpose')
plt.ylabel('Total numbers of Charged off')
plt.tight_layout()
plt.suptitle('home ownership distribution respect to purpose')
plt.grid(True)
plt.legend(loc='upper right')
plt.show()

### Insights
most of chagrged off coming from who taking loan for debt_consolation and renting house.

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=charged_off_main_csv,x='home_ownership',hue='grade')
plt.ylabel('Total numbers of Charged off')
plt.tight_layout()
plt.suptitle('home ownership distribution respect to grade')
plt.grid(True)
plt.legend(loc='upper right')
plt.show()

### Insights
Persons who leaving in rent and grade having C B D  E are have most no. of Charged Off

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=charged_off_main_csv,x='term',hue='grade')
plt.ylabel('Total numbers of Charged off')
plt.tight_layout()
plt.suptitle('term distribution respect to grade')
plt.grid(True)
plt.legend(loc='upper right')
plt.show()

### Insights
- Mostly charged off for 36 months grade C and B
- Mostly charged off for 60 months grade C and D

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=charged_off_main_csv,x='home_ownership',hue='term')
plt.ylabel('Total numbers of Charged off')
plt.tight_layout()
plt.suptitle('home ownership distribution respect to term')
plt.grid(True)
plt.legend(loc='upper right')
plt.show()

### Insights
Mostly charged off for rent 36 monts and leaving on Mortage

In [ ]:
charged_off_main_csv.columns

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=charged_off_main_csv,x='annual_inc_cats',hue='home_ownership')
plt.ylabel('Total numbers of Charged off')
plt.tight_layout()
plt.suptitle('annual_inc_cats distribution respect to home_ownership')
plt.grid(True)
plt.legend(loc='upper right')
plt.show()

### Insights
No. of pepoles Charged _off which annual_income have 20k to 60k which on living on rent

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=charged_off_main_csv,x='int_rate_cats',hue='home_ownership')
plt.ylabel('Total numbers of Charged off')
plt.tight_layout()
plt.suptitle('int_rate_cats distribution respect to term')
plt.grid(True)
plt.legend(loc='upper right')
plt.show()

### Insights
No. of majority pepole charged_off which one taking loan on 12.4 to 16 interest rate and leaving on rent.

# Bivariate Analysis
taking loan amont with all varables

In [ ]:
plt.figure(figsize=(15,4))
plt.subplot(1,2,1)
sns.boxplot(x="int_rate_cats", y="loan_amnt", data=main_csv)
plt.title("Interest Rate vs loan amount")
plt.subplot(1,2,2)
sns.boxplot(x="verification_status", y="loan_amnt", data=main_csv)
plt.title("Verification Status vs loan amount ")
plt.show()

### Insights
- Interest rate have higher on higher loan amount
- Verified pepole got higher loan amount respect to others

In [ ]:
plt.figure(figsize=(15,7))
plt.subplot(1,2,1)
sns.boxplot(x="year", y="loan_amnt", data=main_csv)
plt.title("years vs loan amount")

plt.subplot(1,2,2)
sns.boxplot(x="month", y="loan_amnt", data=main_csv)
plt.title("months vs loan amount ")
plt.show()

### Insights
- loan amount increasing year by year
- First of the month borrower geting higher loan amount

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.boxplot(x="home_ownership", y="loan_amnt", data=main_csv)
plt.title("home_ownership vs loan amount ")
plt.figure(figsize=(10,7))
plt.subplot(1,2,2)
sns.boxplot(y="purpose", x="loan_amnt", data=main_csv)
plt.title("  loan amount vs purpose")
plt.show()

### Insights
- Mortage persons are get higher amount respect to other categories of home ownership.
- Small business alloted higher loan amount respect to others

In [ ]:
plt.figure(figsize=(15,7))
plt.subplot(1,2,1)
sns.boxplot(x="term", y="loan_amnt", data=main_csv)
plt.title("term vs loan amount ")

plt.subplot(1,2,2)
sns.boxplot(x="grade", y="loan_amnt", data=main_csv)
plt.title("grade vs loan amount ")
plt.show()

### Insights
Persons who applied loan for 60 month term got hiagher loan amount
Grade G and F got higher loan amount.

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(1,2,1)
sns.boxplot(y="sub_grade", x="loan_amnt", data=main_csv)
plt.title(" loan amount vs sub_grade")



plt.subplot(1,2,2)
sns.boxplot(x="emp_length", y="loan_amnt", data=main_csv)
plt.title("emp_length vs loan amount ")


plt.show()

### Insights
- Sub Grade G and F got higher loan amount its following Grades
- Persons who have more then 10 years experience got higher loan amount

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(1,1,1)
sns.boxplot(x="loan_status", y="loan_amnt", data=main_csv)
plt.title(" loan_status vs loan_amnt")
plt.show()

### Insights
Boroworrs who got charged_off alloted higher loan amount

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(1,1,1)
sns.boxplot(x="annual_inc_cats", y="loan_amnt", data=main_csv)
plt.title(" annual_inc_cats vs loan_amnt")
plt.show()

### Insights
Boroworrs which one have higher annual income got alloted higher loan amount

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(1,1,1)
sns.boxplot(x="dti_cats", y="loan_amnt", data=main_csv)
plt.title(" dti_cats vs loan_amnt")
plt.show()

### Insights
Boroworrs which one have higher dti rate got alloted higher loan amount

# Bivariate Analsys part 2

Now doing analysis with charged_off prportion with each variables

In [ ]:
# Derived columns
purpose_vs_loan_status = main_csv.groupby(['purpose', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
purpose_vs_loan_status['Total'] = purpose_vs_loan_status['Charged Off'] + purpose_vs_loan_status['Fully Paid'] 
purpose_vs_loan_status['Chargedoff_Proportion'] = purpose_vs_loan_status['Charged Off'] / purpose_vs_loan_status['Total']

In [ ]:
purpose_vs_loan_status

In [ ]:
plt.figure(figsize=(15,8))
plt.barh(y=purpose_vs_loan_status['purpose'],width=purpose_vs_loan_status['Chargedoff_Proportion'],data=purpose_vs_loan_status)
plt.title('loan issued purpose_vs Chargedoff_Proportion count plot',fontsize=14)
plt.ylabel('purpose',fontsize=14)
plt.xlabel('Chargedoff_Proportion',fontsize=14)
plt.grid(True)
plt.show()

### Insights
- Boroworrs which one taking loan for small business having 29.97% chances of charged_off

- Boroworrs which one taking loan for wedding, major_purchases and car have low risk of charged_off.

In [ ]:
# Derived columns
annual_inc_cats_vs_loan_status = main_csv.groupby(['annual_inc_cats', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
annual_inc_cats_vs_loan_status['Total'] = annual_inc_cats_vs_loan_status['Charged Off'] + annual_inc_cats_vs_loan_status['Fully Paid'] 
annual_inc_cats_vs_loan_status['Chargedoff_Proportion'] = annual_inc_cats_vs_loan_status['Charged Off'] / annual_inc_cats_vs_loan_status['Total']
annual_inc_cats_vs_loan_status

In [ ]:
plt.figure(figsize=(15,8))
plt.bar(x=annual_inc_cats_vs_loan_status['annual_inc_cats'],height=annual_inc_cats_vs_loan_status['Chargedoff_Proportion'],data=annual_inc_cats_vs_loan_status)
plt.title(' annual_inc_cats vs Chargedoff_Proportion count plot',fontsize=14)
plt.xlabel('annual_inc_cats',fontsize=14)
plt.ylabel('Chargedoff_Proportion',fontsize=14)
plt.grid(True)
plt.show()

### Insights
Boroworrs which one taking loan have lower income (0 to 20K) Have 24% risk of charged off

In [ ]:
# Derived columns
loan_amnt_vs_loan_status = main_csv.groupby(['loan_amnt_cats', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
loan_amnt_vs_loan_status['Total'] = loan_amnt_vs_loan_status['Charged Off'] + loan_amnt_vs_loan_status['Fully Paid'] 
loan_amnt_vs_loan_status['Chargedoff_Proportion'] = loan_amnt_vs_loan_status['Charged Off'] / loan_amnt_vs_loan_status['Total']
loan_amnt_vs_loan_status

In [ ]:
plt.figure(figsize=(15,8))
plt.bar(x=loan_amnt_vs_loan_status['loan_amnt_cats'],height=loan_amnt_vs_loan_status['Chargedoff_Proportion'],data=loan_amnt_vs_loan_status)
plt.title(' loan_amnt_cats vs Chargedoff_Proportion count plot',fontsize=14)
plt.xlabel('loan_amnt_cats',fontsize=14)
plt.ylabel('Chargedoff_Proportion',fontsize=14)
plt.grid(True)
plt.show()

### Insights
- Boroworrs which one taking loan of higher amount (more then 28K) have 25.75% of charged_off

- Boroworrs which one taking loan of less amount(0 to 7K)have less risk of charged_off

In [ ]:
# Derived columns
addr_state_vs_loan_status = main_csv.groupby(['addr_state', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
addr_state_vs_loan_status['Total'] = addr_state_vs_loan_status['Charged Off'] + addr_state_vs_loan_status['Fully Paid'] 
addr_state_vs_loan_status['Chargedoff_Proportion'] = addr_state_vs_loan_status['Charged Off'] / addr_state_vs_loan_status['Total']
addr_state_vs_loan_status

In [ ]:
plt.figure(figsize=(25,8))
plt.bar(x=addr_state_vs_loan_status['addr_state'],height=addr_state_vs_loan_status['Chargedoff_Proportion'],data=addr_state_vs_loan_status)
plt.title(' addr_state vs Chargedoff_Proportion count plot',fontsize=14)
plt.xlabel('addr_state',fontsize=14)
plt.ylabel('Chargedoff_Proportion',fontsize=14)
plt.grid(True)
plt.show()

### Insights
- Boroworrs which one taking loan coming from MS state have 25% of chances charged_off

- Boroworrs which one taking loan coming from OR DC and ME have low rsik of charged_off

# Multivariate Analysis

In [ ]:
#finding correlation matrix
loan_correlation = main_csv.select_dtypes('number').corr()
loan_correlation

In [ ]:
#plotting correaltion matrix on clustermap
sns.clustermap(loan_correlation,annot=True,figsize=(14, 8))
plt.title("Correlation between Columns \n")
plt.show()

### Insights
Loan amount funded_amt_inv installment fundend_amt are highly postive corelated and dti and annual_inc highly negative coreleated.

# Suggestions
Lending club shuold consider to accepting more loans request less then 10% interest rate as their probabity of charged_off is have lesser.

Lending club shuold consider to accepting more loans request for borrowers which owning house their have lees probabilty to Charged_off.

Lending club shuold consider to accepting more loans request for grade A respect to B C D becuase in B C D E have lots off Charged_off

Lending club shuold consider to accepting more loans request of borrowers which have annual income gretaer then 90000 its charged_off % have nearest 10%.

Lending club need to accept more loan for 36 month term there have less chances of charged_off and it continuously increasing year by year.

Lending club need to accept loan which owning house, annual income have greater then 90000, interest rate is less then 10%.This combination is have less chances to charged_off.

Lending club should accept more loan request of borrowers which one have source verified or verified. Their have less chance of charged_off.

Lending club should accept more loan request of borrowers which one have employment experienc less then 10 years.

Lending club should accept more loan for applied for wedding, major_purchase, car and credit_card here have chances of charged_off is less nearest 10% but for small business have 27.97% charged_off.

Lenading club should accept the loan request for small loan amount For loan amount till 14K charged_off(nearest 13%) have low risk

Lenading club should accept the loan request for coming from addres state IN IA and ME here having low risk of charged off.

In [ ]:
predictors_df = pd.read_pickle("analysis-predictors.bz2", compression='bz2')
target_df = pd.read_pickle("analysis-target.bz2", compression='bz2')

In [ ]:
predictors_df.head()

In [ ]:
target_df.head()

In [ ]:
target_df